In [42]:
from scipy.linalg import expm
import numpy as np
from numpy import polyfit, poly1d
from numpy import ndarray, matrix, complexfloating
import plotly.graph_objects as go
import random
from tqdm import tqdm
from scipy.optimize import leastsq
import datetime


In [43]:
SigmaZ: ndarray = np.array([[1, 0], [0, -1]])
SigmaY: ndarray = np.array([[0, -1j], [1j, 0]])


parameter setting


In [44]:
OmegaQ: float = 2 * np.pi * 5000
OmegaD: float = 2 * np.pi * 5000 + 2 * np.pi * 1  # 这个频率是角频率
# a: complexfloating = 1 / np.sqrt(2) * 1
# b: complexfloating = 1 / np.sqrt(2)
a: complexfloating = 1
b: complexfloating = 0
# b: complexfloating = (100j) / np.sqrt(10000)
A: float = 30 * 2 * np.pi * 1
PhiD: float = np.pi / 2 * 0


In [45]:
initialState: ndarray = np.array([[a], [b]])


In [46]:
def CalculateHamiltonian(OmegaQ: float, OmegaD: float, PhiD: float, A: float, initialState: ndarray, t: float):
    delta: float = OmegaD - OmegaQ
    H: ndarray = np.array([[0.5 * delta, 0.5 * A * np.exp(1j * PhiD)],
                           [0.5 * A * np.exp(-1j * PhiD), -0.5 * delta]])
    tempMatrix: ndarray = -1j * H * t
    timeState: ndarray = expm(tempMatrix) @ initialState
    return timeState


In [47]:
def RabiOscillation(State: ndarray, chooser: int):
    if chooser == 0:
        probability: float = np.abs((State[0][0]))**2
        return probability
    else:
        probability: float = np.abs(State[1][0])**2
        return probability


# Time Rabi


In [48]:
data = []
stateData = []
τ = 0.3
for t in range(0, int(τ * 1000)):
    td = t / 1000
    currentState: ndarray = CalculateHamiltonian(
        OmegaQ=OmegaQ, OmegaD=OmegaD, PhiD=PhiD, A=A, initialState=initialState, t=td)
    tempMatrix: matrix = matrix(currentState).T
    stateData.append(tempMatrix.tolist())
    data.append([td, RabiOscillation(State=currentState, chooser=0)])
nddata = np.array(data)
ndStateData = np.array(stateData)


In [49]:
fig = go.Figure()

fig.add_trace(go.Scatter(x=nddata[:, 0], y=nddata[:, 1], mode='markers'))

fig.update_yaxes(
    range=[0, 1.0],
    dtick=0.5,
)


fig.show()


In [50]:
coo = []

for states in ndStateData:
    a: complexfloating = states[0][0]
    b: complexfloating = states[0][1]
    alpha: float = np.angle(a)
    beta: float = np.angle(b)
    phi: float = beta - alpha
    theta: float = 2 * np.arccos(np.abs(a))
    x: float = np.sin(theta) * np.cos(phi)
    y: float = np.sin(theta) * np.sin(phi)
    z: float = np.cos(theta)
    coo.append([x, y, z])
ndCoo: ndarray = np.array(coo)


In [51]:
u = np.linspace(0, 2 * np.pi, 100)
v = np.linspace(0, np.pi, 100)

x = np.outer(np.cos(u), np.sin(v))
y = np.outer(np.sin(u), np.sin(v))
z = np.outer(np.ones(np.size(u)), np.cos(v))

fig = go.Figure(data=[go.Surface(x=x, y=y, z=z,
                                 opacity=0.4,
                                 colorscale=[[0, 'rgb(176, 226, 243)'], [
                                     1, 'rgb(176, 226, 243)']],
                                 colorbar=dict(
                                 ),
                                 )])


fig.add_trace(
    go.Scatter3d(
        x=ndCoo[:, 0],
        y=ndCoo[:, 1],
        z=ndCoo[:, 2],
        mode='markers',
        marker=dict(color='red', size=4,)
    )
)


fig.update_layout(
    scene=dict(
        xaxis=dict(
            range=[-1.3, 1.3],
            dtick=0.5,
            showbackground=True,
            backgroundcolor="rgb(200, 200, 230)",
        ),
        yaxis=dict(
            range=[-1.3, 1.3],
            dtick=0.5,
            showbackground=True,
            backgroundcolor="rgb(230, 200, 230)",
        ),
        zaxis=dict(
            range=[-1.3, 1.3],
            dtick=0.5,
            showbackground=True,
            backgroundcolor="rgb(230, 230, 200)",
        ),
    ),
    scene_aspectmode='cube',
    width=800,
    height=800,
)

fig.show()


# Ramsey Oscillation


In [52]:
def CalculateHamiltonianNoDrive(OmegaQ: float, OmegaD: float, initialState: ndarray, τ: float):
    delta: float = OmegaD - OmegaQ
    H: ndarray = np.array([[0.5 * delta, 0], [0, -0.5 * delta]])
    tempMatrix: ndarray = -1j * H * τ
    timeState: ndarray = expm(tempMatrix) @ initialState
    return timeState
    

Apply a $R_x(\frac{\pi}{2})$ on initial state, after τ, apply a $R_x(\frac{\pi}{2})$ on the second state.


In [54]:
def CalculateRamseyFinalState(OmegaQ: float, OmegaD: float, PhiD: float, A: float, initialState: ndarray, τ: float):
    SigmaX: matrix = np.matrix([[0, 1], [1, 0]])
    SigmaZ: ndarray = np.array([[1, 0], [0, -1]])
    SigmaY: ndarray = np.array([[0, -1j], [1j, 0]])
    delta: float = OmegaD - OmegaQ
    T: float = (2 * np.pi) / (2 * 0.5 * np.sqrt(A**2))
    t_1: float = T / 4
    firstState: ndarray = CalculateHamiltonian(
        OmegaQ, OmegaD, PhiD, A, initialState, t_1)
    secondState: ndarray = CalculateHamiltonianNoDrive(
        OmegaQ, OmegaD, firstState, τ)

    meanX: matrix = np.real(
        np.matrix(secondState).T.conjugate() @ SigmaX @ np.matrix(secondState))

    finalState: ndarray = CalculateHamiltonian(
        OmegaQ, OmegaD, PhiD, A, secondState, t_1)
    return [finalState, meanX]


In [55]:
testlist = CalculateRamseyFinalState(OmegaQ, OmegaD, PhiD, A, initialState, 10)
print(testlist[1][0, 0])
print(testlist[0])


0.033325385874853945
[[-0.00087242-0.03331482j]
 [ 0.        -0.99944453j]]


During delay time τ: Easy Version


In [56]:
τ: float = 5
data = []
stateData = []
meanXData = []
for t in range(0, τ * 1000 + 1):
    td = t / 1000
    calculatedResult: list = CalculateRamseyFinalState(
        OmegaQ, OmegaD, PhiD, A, initialState, td)
    currentState: ndarray = calculatedResult[0]
    currentMeanX: float = calculatedResult[1][0, 0]
    tempMatrix: matrix = matrix(currentState).T
    stateData.append(tempMatrix.tolist())
    data.append([td, RabiOscillation(State=currentState, chooser=0)])
    meanXData.append([td, currentMeanX])
nddata = np.array(data)
ndStateData = np.array(stateData)
ndMeanXData = np.array(meanXData)


In [57]:
# |0> probability
fig = go.Figure()

fig.add_trace(go.Scatter(x=nddata[:, 0], y=nddata[:, 1], mode='markers'))

fig.update_yaxes(
    range=[0, 1.0],
    dtick=0.5,
)

fig.show()


频率是 $n * 2π$ 的 n <- 此乃频率，角频率为 $2πn$

$δ = 0$的时候周期无限大故无振荡


In [58]:
# Mean σX: <ψ|σX|ψ>
fig = go.Figure()

fig.add_trace(go.Scatter(
    x=ndMeanXData[:, 0], y=ndMeanXData[:, 1], mode='markers'))

fig.update_yaxes(
    range=[-1, 1],
    dtick=0.5,
)

fig.show()


In [60]:
coo = []

for states in tqdm(ndStateData, desc="Calculating ... ", colour="red"):
    a: complexfloating = states[0][0]
    b: complexfloating = states[0][1]
    alpha: float = np.angle(a)
    beta: float = np.angle(b)
    phi: float = beta - alpha
    theta: float = 2 * np.arccos(np.abs(a))
    x: float = np.sin(theta) * np.cos(phi)
    y: float = np.sin(theta) * np.sin(phi)
    z: float = np.cos(theta)
    coo.append([x, y, z])
ndCoo: ndarray = np.array(coo)


Calculating ... : 100%|██████████| 5001/5001 [00:00<00:00, 96196.37it/s]


In [61]:
u = np.linspace(0, 2 * np.pi, 100)
v = np.linspace(0, np.pi, 100)

x = np.outer(np.cos(u), np.sin(v))
y = np.outer(np.sin(u), np.sin(v))
z = np.outer(np.ones(np.size(u)), np.cos(v))

fig = go.Figure(data=[go.Surface(x=x, y=y, z=z,
                                 opacity=0.4,
                                 colorscale=[[0, 'rgb(176, 226, 243)'], [
                                     1, 'rgb(176, 226, 243)']],
                                 colorbar=dict(
                                 ),
                                 )])


fig.add_trace(
    go.Scatter3d(
        x=ndCoo[:, 0],
        y=ndCoo[:, 1],
        z=ndCoo[:, 2],
        mode='markers',
        marker=dict(color='red', size=4,)
    )
)


fig.update_layout(
    scene=dict(
        xaxis=dict(
            range=[-1.3, 1.3],
            dtick=0.5,
            showbackground=True,
            backgroundcolor="rgb(200, 200, 230)",
        ),
        yaxis=dict(
            range=[-1.3, 1.3],
            dtick=0.5,
            showbackground=True,
            backgroundcolor="rgb(230, 200, 230)",
        ),
        zaxis=dict(
            range=[-1.3, 1.3],
            dtick=0.5,
            showbackground=True,
            backgroundcolor="rgb(230, 230, 200)",
        ),
    ),
    scene_aspectmode='cube',
    width=800,
    height=800,
)

fig.show()


During delay time τ: Complex Version


In [62]:
τ: float = 5
ndMeasurementData = np.array([[0, 0]])
precision = 15
measureTimes = 1000
for t in tqdm(range(0, τ * precision + 1), desc="Calculating ... ", colour="#6362fa"):
    td = t / precision
    counterForState0: int = 0
    for i in range(0, measureTimes):
        OmegaQ_Modified = OmegaQ + np.random.normal(0, 0.8)
        calculatedState: ndarray = CalculateRamseyFinalState(
            OmegaQ_Modified, OmegaD, PhiD, A, initialState, td)[0]
        probability0: float = (np.abs(calculatedState[0][0]))**2
        numRandom: float = random.random()
        if numRandom <= probability0:
            counterForState0 = counterForState0 + 1
    probabilityThisMeasurement = counterForState0 / measureTimes
    ndMeasurementData = np.r_[ndMeasurementData,
                              [[td, probabilityThisMeasurement]]]


Calculating ... : 100%|██████████| 76/76 [00:38<00:00,  1.98it/s]


In [63]:
ndMeasurementData[1:, 0]


array([0.        , 0.06666667, 0.13333333, 0.2       , 0.26666667,
       0.33333333, 0.4       , 0.46666667, 0.53333333, 0.6       ,
       0.66666667, 0.73333333, 0.8       , 0.86666667, 0.93333333,
       1.        , 1.06666667, 1.13333333, 1.2       , 1.26666667,
       1.33333333, 1.4       , 1.46666667, 1.53333333, 1.6       ,
       1.66666667, 1.73333333, 1.8       , 1.86666667, 1.93333333,
       2.        , 2.06666667, 2.13333333, 2.2       , 2.26666667,
       2.33333333, 2.4       , 2.46666667, 2.53333333, 2.6       ,
       2.66666667, 2.73333333, 2.8       , 2.86666667, 2.93333333,
       3.        , 3.06666667, 3.13333333, 3.2       , 3.26666667,
       3.33333333, 3.4       , 3.46666667, 3.53333333, 3.6       ,
       3.66666667, 3.73333333, 3.8       , 3.86666667, 3.93333333,
       4.        , 4.06666667, 4.13333333, 4.2       , 4.26666667,
       4.33333333, 4.4       , 4.46666667, 4.53333333, 4.6       ,
       4.66666667, 4.73333333, 4.8       , 4.86666667, 4.93333

In [64]:
fig = go.Figure()

fig.add_trace(go.Scatter(
    x=ndMeasurementData[1:, 0], y=ndMeasurementData[1:, 1], mode='markers'))

fig.update_yaxes(
    range=[-0.1, 1.1],
    dtick=0.5,
)


fig.show()


Fit the curve


In [65]:
def function(t, p):
    a, T2, omega, phi, b = p
    return a*np.exp(-(t/T2)**2)*np.cos(omega*t+phi)+b

# define noise function


def f_err(p, y, t):
    return y - function(t, p)


In [66]:
c, ret_val = leastsq(f_err, [1, 1, 1, 1, 1], args=(
    ndMeasurementData[1:, 1], ndMeasurementData[1:, 0]))
print(c, ret_val)


[-0.49762338  1.74829827  6.30704583  0.06195804  0.49848476] 1


In [67]:
fig = go.Figure()

fig.add_trace(go.Scatter(
    x=ndMeasurementData[1:, 0], y=ndMeasurementData[1:, 1], mode='markers'))

fig.add_trace(
    go.Scatter(
        x=ndMeasurementData[1:, 0],
        y=c[0] * np.exp(-(ndMeasurementData[1:, 0] / (-1.7))**2) *
        np.cos(c[2]*ndMeasurementData[1:, 0]+c[3])+c[4]
    )
)

fig.update_yaxes(
    range=[-0.1, 1.1],
    dtick=0.5,
)


fig.show()


To know the relationship between σ and T2


In [68]:
def CalculateRemseyData(sigma: float):
    τ: float = 5
    ndMeasurementData: ndarray = np.array([[0, 0]])
    precision = 15
    measureTimes = 1000
    for t in range(0, τ * precision + 1):
        td = t / precision
        counterForState0: int = 0
        for i in range(0, measureTimes):
            OmegaQ_Modified = OmegaQ + np.random.normal(0, sigma)
            calculatedState: ndarray = CalculateRamseyFinalState(
                OmegaQ_Modified, OmegaD, PhiD, A, initialState, td)[0]
            probability0: float = (np.abs(calculatedState[0][0]))**2
            numRandom: float = random.random()
            if numRandom <= probability0:
                counterForState0 = counterForState0 + 1
        probabilityThisMeasurement = counterForState0 / measureTimes
        ndMeasurementData = np.r_[ndMeasurementData,
                                  [[td, probabilityThisMeasurement]]]
    return ndMeasurementData


In [ ]:
sigmaRange: int = 5
precisionSigma: int = 10
sigmaT2Data: ndarray = np.array([[0, 0]])
for sigma in tqdm(range(0, sigmaRange * precisionSigma + 1)):
    sigmad = sigma / precisionSigma
    measurementData: ndarray = CalculateRemseyData(sigmad)
    c, ret_val = leastsq(f_err, [1, 1, 1, 1, 1], args=(
        measurementData[1:, 1], measurementData[1:, 0]))
    sigmaT2Data = np.r_[sigmaT2Data, [[sigmad, np.abs(c[1])]]]


#### Save Data


In [ ]:
np.save(f'./data/sigma-T2-{datetime.date.today()}_1', sigmaT2Data)


#### Load Data


In [69]:
sigmaT2Data = np.load('./data/sigma-T2-2023-04-18_1.npy')


In [76]:
fig = go.Figure()

fig.add_trace(go.Scatter(
    x=sigmaT2Data[2:, 0], y=sigmaT2Data[2:, 1], mode='markers'))

fig.update_yaxes(
    dtick=5,
)


fig.show()


# Spectroscopy


In [74]:
data = []
A_2 = 1 * 2 * np.pi
for omegaD in tqdm(range(int(4800 * 2 * np.pi), int(5200 * 2 * np.pi)), desc="Calculating...", colour="green"):
    currentState = CalculateHamiltonian(
        OmegaQ, omegaD, PhiD, A_2, initialState, 2)
    probability0 = (np.abs(currentState[0][0]))**2
    data.append([omegaD / (2 * np.pi), probability0])
ndData = np.array(data)


Calculating...: 100%|██████████| 2513/2513 [00:00<00:00, 6968.27it/s]


In [75]:
fig = go.Figure()

fig.add_trace(go.Scatter(x=ndData[:, 0], y=ndData[:, 1], mode='markers'))

fig.update_yaxes(
    range=[-0.1, 1.1],
    dtick=0.5,
)


fig.show()
